Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using DynamicHMCModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_d("..", "scripts", "10")
cd(ProjDir)

### snippet 10.4

In [3]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);
df[:pulled_left] = convert(Array{Int64}, df[:pulled_left])
df[:prosoc_left] = convert(Array{Int64}, df[:prosoc_left])
df[:condition] = convert(Array{Int64}, df[:condition])
df[:actor] = convert(Array{Int64}, df[:actor])
first(df, 5)

struct m_10_04d_model{TY <: AbstractVector, TX <: AbstractMatrix,
  TA <: AbstractVector}
    "Observations."
    y::TY
    "Covariates"
    X::TX
    "Actors"
    A::TA
    "Number of observations"
    N::Int
    "Number of unique actors"
    N_actors::Int
end

Make the type callable with the parameters *as a single argument*.

In [4]:
function (problem::m_10_04d_model)(θ)
    @unpack y, X, A, N, N_actors = problem   # extract the data
    @unpack β, α = θ  # works on the named tuple too
    ll = 0.0
    ll += sum(logpdf.(Normal(0, 10), β)) # bp & bpC
    ll += sum(logpdf.(Normal(0, 10), α)) # alpha[1:7]
    ll += sum(
      [loglikelihood(Binomial(1, logistic(α[A[i]] + dot(X[i, :], β))), [y[i]]) for i in 1:N]
    )
    ll
end

Instantiate the model with data and inits.

In [5]:
N = size(df, 1)
N_actors = length(unique(df[:actor]))
X = hcat(ones(Int64, N), df[:prosoc_left] .* df[:condition]);
A = df[:actor]
y = df[:pulled_left]
p = m_10_04d_model(y, X, A, N, N_actors);
θ = (β = [1.0, 0.0], α = [-1.0, 10.0, -1.0, -1.0, -1.0, 0.0, 2.0])
p(θ)

-305.21943396408915

Write a function to return properly dimensioned transformation.

In [6]:
problem_transformation(p::m_10_04d_model) =
    as( (β = as(Array, size(p.X, 2)), α = as(Array, p.N_actors), ) )
# Wrap the problem with a transformation, then use Flux for the gradient.
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));

Tune and sample.

In [7]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.014 s/step ...done
MCMC, adapting ϵ (25 steps)
0.016 s/step ...done
MCMC, adapting ϵ (50 steps)
0.008 s/step ...done
MCMC, adapting ϵ (100 steps)
0.0069 s/step ...done
MCMC, adapting ϵ (200 steps)
0.0041 s/step ...done
MCMC, adapting ϵ (400 steps)
step 284 (of 400), 0.0035 s/step
0.0035 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0037 s/step ...done
MCMC (1000 steps)
step 281 (of 1000), 0.0036 s/step
step 561 (of 1000), 0.0036 s/step
step 866 (of 1000), 0.0035 s/step
0.0035 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [8]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));
posterior[1:5]

5-element Array{NamedTuple{(:β, :α),Tuple{Array{Float64,1},Array{Float64,1}}},1}:
 (β = [2.1944985925210947, 0.5901977005585288], α = [-2.3926535766120374, 5.802705404915738, -2.950189262346768, -2.9150851455677063, -2.6977874340833163, -1.839972629721614, 0.47047982990272263])  
 (β = [-5.905554031337963, 0.2406890376219152], α = [5.320746953333941, 12.431171815418521, 5.40778362767297, 5.059491951845445, 5.51238210459692, 7.166498986075312, 7.83464725980286])              
 (β = [0.38972647504114, 0.5067682071495101], α = [-0.5070679176878703, 5.548019777372892, -1.2161907063209385, -0.8326685420438382, -1.071505726283445, -0.29395288669164543, 1.2570080736919313])   
 (β = [2.7122386387450126, 0.7084000766958038], α = [-3.187036388096761, 4.7820009709932885, -3.7490960156409465, -3.2397965649553098, -2.9482761801846533, -2.3729816812187967, -0.9000369978376922])
 (β = [3.388074071809994, 0.9786431923082679], α = [-4.060778761623799, 7.625242889306762, -4.7697233348867165, -4.4398202

Extract the parameter posterior means: `β`,

In [9]:
posterior_β = mean(first, posterior)

2-element Array{Float64,1}:
 1.6533702104616024 
 0.41780660138071435

Extract the parameter posterior means: `β`,

In [10]:
posterior_α = mean(last, posterior)

7-element Array{Float64,1}:
 -2.1033239879990586 
  9.926714307940099  
 -2.406324705547061  
 -2.395967884577413  
 -2.1052963437255325 
 -1.1741111112175675 
  0.38779985324948246

Effective sample sizes (of untransformed draws)

In [11]:
ess = mapslices(effective_sample_size, get_position_matrix(chain); dims = 1)
ess

1×9 Array{Float64,2}:
 622.566  1000.0  619.31  227.917  631.93  621.885  624.867  623.517  624.972

NUTS-specific statistics

In [12]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.85, min/25%/median/75%/max: 0.0 0.82 0.88 0.92 0.97
  termination: AdjacentDivergent => 1% AdjacentTurn => 25% DoubledTurn => 75%
  depth: 2 => 2% 3 => 42% 4 => 55% 5 => 1%


Result rethinking

In [13]:
rethinking = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
        Mean        SD       Naive SE       MCSE      ESS
a.1 -0.74503184 0.26613979 0.0042080396 0.0060183398 1000
a.2 10.77955494 5.32538998 0.0842018089 0.1269148045 1000
a.3 -1.04982353 0.28535997 0.0045119373 0.0049074219 1000
a.4 -1.04898135 0.28129307 0.0044476339 0.0056325117 1000
a.5 -0.74390933 0.26949936 0.0042611590 0.0052178124 1000
a.6  0.21599365 0.26307574 0.0041595927 0.0045153523 1000
a.7  1.81090866 0.39318577 0.0062168129 0.0071483527 1000
 bp  0.83979926 0.26284676 0.0041559722 0.0059795826 1000
bpC -0.12913322 0.29935741 0.0047332562 0.0049519863 1000
";

Means of draws

In [14]:
[posterior_β, posterior_α]

2-element Array{Array{Float64,1},1}:
 [1.6533702104616024, 0.41780660138071435]                                                                                                      
 [-2.1033239879990586, 9.926714307940099, -2.406324705547061, -2.395967884577413, -2.1052963437255325, -1.1741111112175675, 0.38779985324948246]

End of `10/m10.04d.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*